In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sorting import standardise, unstandardise, k_fold_splits
from linear_regressions import prediction_error, linear_data, linear_regression, linear_analysis
from polyn_regression import poly_data, poly_regression, poly_analysis
from ridge_regressions import ridge_regression, pre_poly, ridge_analysis
from gd_linear import energy_function1, gradient_descent1
from gd_ridge import energy_function2, gradient_descent2
from gd_huber import huber_gradient, energy_function3, huber_loss_gradient_descent
from proximal_gd import proximal_map, proximal_gradient_descent

In [ ]:
fires = pd.read_csv("forestfires.csv")

months = {"jan": 1., "feb": 2., "mar": 3., "apr": 4., "may": 5., "jun": 6., "jul": 7., "aug": 8., "sep": 9., "oct": 10., "nov": 11., "dec": 12.}
days = {"mon": 1., "tue": 2., "wed": 3., "thu": 4., "fri": 5., "sat": 6., "sun": 7.}

fires["month"] = fires["month"].map(months)
fires["day"] = fires["day"].map(days)

fires["ln_area"] = np.log(fires["area"] + 1)

In [ ]:
corr = fires[["X", "Y", "month", "day", "FFMC", "DMC", "DC", "ISI", "temp", "RH", "wind", "rain", "area"]].corr()

fig, axes = plt.subplots(figsize = (10, 3.5))

sns.heatmap(corr, cmap = "viridis", annot = True, fmt = ".2f", ax = axes)

plt.tight_layout()
plt.show()

In [ ]:
sizes = fires.groupby(["X", "Y"])["area"].mean().to_dict()
x = [i[0] for i in sizes.keys()]
y = [i[1] for i in sizes.keys()]
means = [((i * 100)+1) for i in sizes.values()]

plt.figure(figsize = (10, 3))

plt.scatter(x, y, s = means, alpha = 0.5, c = "springgreen", ec = "black")
plt.xlabel("X Spatial Coordinate")
plt.ylabel("Y Spatial Coordinate")
plt.grid(alpha = 0.1)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 6, figsize = (20, 7.5))

axes[0, 0].hist(fires["month"], bins = 12, ec = "black", color = "cornflowerblue")
axes[0, 0].set_title("Months of the Year")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].set_xlabel("Month")

axes[0, 1].hist(fires["day"], bins = 7, ec = "black", color = "cornflowerblue")
axes[0, 1].set_title("Days of the Week")
axes[0, 1].set_xlabel("Day")

axes[0, 2].hist(fires["FFMC"], bins = 30, ec = "black", color = "cornflowerblue")
axes[0, 2].set_title("Fine Fuel Moisture Code")
axes[0, 2].set_xlabel("FFMC")

axes[0, 3].hist(fires["DMC"], bins = 30, ec = "black", color = "cornflowerblue")
axes[0, 3].set_title("Duff Moisture Code")
axes[0, 3].set_xlabel("DMC")

axes[0, 4].hist(fires["DC"], bins = 30, ec = "black", color = "cornflowerblue")
axes[0, 4].set_title("Drought Code")
axes[0, 4].set_xlabel("DC")

axes[0, 5].hist(fires["area"], bins = 30, ec = "black", color = "cornflowerblue")
axes[0, 5].set_title("Area")
axes[0, 5].set_xlabel("HA")

axes[1, 0].hist(fires["ISI"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 0].set_title("Initial Spread Index")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].set_xlabel("ISI")

axes[1, 1].hist(fires["temp"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 1].set_title("Temperature")
axes[1, 1].set_xlabel("Celsius")

axes[1, 2].hist(fires["RH"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 2].set_title("Relative Humidity")
axes[1, 2].set_xlabel("%")

axes[1, 3].hist(fires["wind"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 3].set_title("Wind Speed")
axes[1, 3].set_xlabel("KM/H")

axes[1, 4].hist(fires["rain"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 4].set_title("Rain")
axes[1, 4].set_xlabel("MM/M^2")

axes[1, 5].hist(fires["ln_area"], bins = 30, ec = "black", color = "cornflowerblue")
axes[1, 5].set_title("Logarithm of Area")
axes[1, 5].set_xlabel("Log(HA+1)")

plt.tight_layout()
plt.show()

In [ ]:
months = {b"jan": 1., b"feb": 2., b"mar": 3., b"apr": 4., b"may": 5., b"jun": 6., b"jul": 7., b"aug": 8., b"sep": 9., b"oct": 10., b"nov": 11.,
          b"dec": 12.}
days = {b"mon": 1., b"tue": 2., b"wed": 3., b"thu": 4., b"fri": 5., b"sat": 6., b"sun": 7.}

converter_month = lambda x: months[x]
converter_day = lambda x: days[x]
converter_log = lambda x: np.log(1 + float(x))

forest_fires_data = np.genfromtxt("forestfires.csv", delimiter = ",", skip_header = 1, usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                  converters = {2: converter_month, 3: converter_day, 12: converter_log})

standardised_forest_fires_data, data_mean, data_std = standardise(forest_fires_data)
k_fold_training_data, k_fold_testing_data = k_fold_splits(standardised_forest_fires_data, 5)

In [ ]:
# linear regression

linear_training_data, linear_testing_data = [], []
for i in range(len(k_fold_training_data)):
    linear_training_data.append((linear_data(k_fold_training_data[i][0]), k_fold_training_data[i][1]))
    linear_testing_data.append((linear_data(k_fold_testing_data[i][0]), k_fold_testing_data[i][1]))

linear_analysis(linear_training_data, linear_testing_data)

In [ ]:
# poly regression

poly_analysis(k_fold_training_data, k_fold_testing_data, 50)

In [ ]:
# ridge regression

ridge_training_data, ridge_testing_data = pre_poly(k_fold_training_data, k_fold_testing_data, 25)

optimal_degree, optimal_alpha, best_weights_mean, best_weights_std, best_mse_mean, best_mse_std = ridge_analysis(ridge_training_data,
                                                                                                                 ridge_testing_data,
                                                                                                                 np.linspace(0, 10000, 500))

print(optimal_degree, optimal_alpha, best_weights_mean, best_weights_std, best_mse_mean, best_mse_std)

In [ ]:
# gradient descent - linear

gradient_weights1 = np.zeros(linear_training_data[0][0].shape[1])

step_size1 = np.logspace(-10, -0.4, 51) # -0.3
gd1_weights_list, gd1_mse_list = [], []
for size in step_size1:
    gd1_weights, gd1_mse = gradient_descent1(linear_training_data, linear_testing_data, gradient_weights1, size)
    gd1_weights_list.append(gd1_weights)
    gd1_mse_list.append(gd1_mse)

In [ ]:
# gradient descent - ridge

gradient_weights2 = np.zeros(ridge_training_data[optimal_degree][0][0].shape[1])

step_size2 = np.logspace(-10, -5, 51)
gd2_weights_list, gd2_mse_list = [], []
for size in step_size2:
    gd2_weights, gd2_mse = gradient_descent2(ridge_training_data[optimal_degree], ridge_testing_data[optimal_degree], gradient_weights2,
                                             optimal_alpha, size)
    gd2_weights_list.append(gd2_weights)
    gd2_mse_list.append(gd2_mse)

In [ ]:
# gradient descent with huber-loss lasso

step_size3 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict1 = {}
for size in step_size3:
    for a in alpha:
        for p in paramater:
            gd3_weights, gd3_mse = huber_loss_gradient_descent(linear_training_data, linear_testing_data, gradient_weights1, a, p, size)
            if (a, p) not in mse_dict1:
                mse_dict1[(a, p)] = []
            mse_dict1[(a, p)].append(gd3_mse)

In [ ]:
# proximal gradient descent

step_size4 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict2 = {}
for size in step_size4:
    for a in alpha:
        for p in paramater:
            gd4_weights, gd4_mse = proximal_gradient_descent(linear_training_data, linear_testing_data, gradient_weights1, a, p, size)
            if (a, p) not in mse_dict2:
                mse_dict2[(a, p)] = []
            mse_dict2[(a, p)].append(gd4_mse)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize = (20, 4))

axes[0].plot(step_size1, gd1_mse_list)
axes[0].set_title("Linear Gradient Descent")
axes[0].set_xlabel("Step Size")
axes[0].set_ylabel("MSE")
axes[0].grid(alpha = 0.1)

axes[1].plot(step_size2, gd2_mse_list)
axes[1].set_title("Ridge Gradient Descent")
axes[1].set_xlabel("Step Size")
axes[1].grid(alpha = 0.1)

for key, value in mse_dict1.items():
    axes[2].plot(step_size3, value, label = f"(alpha, paramater):{key}", alpha = 0.5)
axes[2].set_title("Gradient Descent with Huber Loss")
axes[2].set_xlabel("Step Size")
axes[2].grid(alpha = 0.1)
axes[2].legend()

for key, value in mse_dict2.items():
    axes[3].plot(step_size4, value, label = f"alpha:{key}", alpha = 0.5)
axes[3].set_title("Proximal Gradient Descent")
axes[3].set_xlabel("Step Size")
axes[3].grid(alpha = 0.1)
axes[3].legend()

plt.tight_layout()
plt.show()

In [ ]:
def prediction(inputs, weights):
    return inputs@weights

final_weights, final_mse = gradient_descent1(linear_training_data, linear_testing_data, gradient_weights1, 0.05)

fig, axes = plt.subplots(1, len(linear_training_data), figsize = (20, 4))

for i in range(len(linear_training_data)):
    stand_y_predict = prediction(linear_testing_data[i][0], final_weights)
    unstand_y_predict = unstandardise(stand_y_predict, data_mean[12], data_std[12])
    normal_y_predict = np.exp(unstand_y_predict) - 1

    unstand_y_original = unstandardise(linear_testing_data[i][1], data_mean[12], data_std[12])
    normal_y_original = np.exp(unstand_y_original) - 1

    axes[i].plot([j for j in range(normal_y_predict.shape[0])], normal_y_predict, label = "prediction", c = "springgreen")
    axes[i].plot([k for k in range(normal_y_original.shape[0])], normal_y_original, label = "actual", alpha = 0.5, c = "tomato")
    axes[i].legend()

axes[0].set_ylabel("Area")
plt.tight_layout()
plt.show()

print(final_weights, final_mse)

In [ ]:
# second attempt

cycle = np.linspace(0, np.pi, 12)
months2 = {b"jan": 0., b"feb": np.sin(cycle[1]), b"mar": np.sin(cycle[2]), b"apr": np.sin(cycle[3]), b"may": np.sin(cycle[4]),
          b"jun": np.sin(cycle[5]), b"jul": np.sin(cycle[6]), b"aug": np.sin(cycle[7]), b"sep": np.sin(cycle[8]), b"oct": np.sin(cycle[9]),
          b"nov": np.sin(cycle[10]), b"dec": 0.}

converter_month2 = lambda x: months2[x]

forest_fires_data2 = np.genfromtxt("forestfires.csv", delimiter = ",", skip_header = 1, usecols = [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                  converters = {2: converter_month2, 4: converter_log, 6: converter_log, 7: converter_log, 11: converter_log,
                                                12: converter_log})

standardised_forest_fires_data2, data_mean2, data_std2 = standardise(forest_fires_data2)
k_fold_training_data2, k_fold_testing_data2 = k_fold_splits(standardised_forest_fires_data2, 5)

In [ ]:
linear_training_data2, linear_testing_data2 = [], []
for i in range(len(k_fold_training_data2)):
    linear_training_data2.append((linear_data(k_fold_training_data2[i][0]), k_fold_training_data2[i][1]))
    linear_testing_data2.append((linear_data(k_fold_testing_data2[i][0]), k_fold_testing_data2[i][1]))

linear_analysis(linear_training_data2, linear_testing_data2)

In [ ]:
poly_analysis(k_fold_training_data2, k_fold_testing_data2, 50)

In [ ]:
ridge_training_data2, ridge_testing_data2 = pre_poly(k_fold_training_data2, k_fold_testing_data2, 25)

optimal_degree2, optimal_alpha2, best_weights_mean2, best_weights_std2, best_mse_mean2, best_mse_std2 = ridge_analysis(ridge_training_data2,
                                                                                                                       ridge_testing_data2,
                                                                                                                       np.linspace(0, 10000, 500))

print(optimal_degree2, optimal_alpha2, best_weights_mean2, best_weights_std2, best_mse_mean2, best_mse_std2)

In [ ]:
gradient_weights3 = np.zeros(linear_training_data2[0][0].shape[1])

step_size1 = np.logspace(-10, -0.4, 51)
gd1_weights_list, gd1_mse_list = [], []
for size in step_size1:
    gd1_weights, gd1_mse = gradient_descent1(linear_training_data2, linear_testing_data2, gradient_weights3, size)
    gd1_weights_list.append(gd1_weights)
    gd1_mse_list.append(gd1_mse)

In [ ]:
gradient_weights4 = np.zeros(ridge_training_data2[optimal_degree2][0][0].shape[1])

step_size2 = np.logspace(-10, -5, 51)
gd2_weights_list, gd2_mse_list = [], []
for size in step_size2:
    gd2_weights, gd2_mse = gradient_descent2(ridge_training_data2[optimal_degree2], ridge_testing_data2[optimal_degree2], gradient_weights4,
                                             optimal_alpha2, size)
    gd2_weights_list.append(gd2_weights)
    gd2_mse_list.append(gd2_mse)

In [ ]:
step_size3 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict1 = {}
for size in step_size3:
    for a in alpha:
        for p in paramater:
            gd3_weights, gd3_mse = huber_loss_gradient_descent(linear_training_data2, linear_testing_data2, gradient_weights3, a, p, size)
            if (a, p) not in mse_dict1:
                mse_dict1[(a, p)] = []
            mse_dict1[(a, p)].append(gd3_mse)

In [ ]:
step_size4 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict2 = {}
for size in step_size4:
    for a in alpha:
        for p in paramater:
            gd4_weights, gd4_mse = proximal_gradient_descent(linear_training_data2, linear_testing_data2, gradient_weights3, a, p, size)
            if (a, p) not in mse_dict2:
                mse_dict2[(a, p)] = []
            mse_dict2[(a, p)].append(gd4_mse)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize = (20, 4))

axes[0].plot(step_size1, gd1_mse_list)
axes[0].set_title("Linear Gradient Descent")
axes[0].set_xlabel("Step Size")
axes[0].set_ylabel("MSE")
axes[0].grid(alpha = 0.1)

axes[1].plot(step_size2, gd2_mse_list)
axes[1].set_title("Ridge Gradient Descent")
axes[1].set_xlabel("Step Size")
axes[1].grid(alpha = 0.1)

for key, value in mse_dict1.items():
    axes[2].plot(step_size3, value, label = f"(alpha, paramater):{key}", alpha = 0.5)
axes[2].set_title("Gradient Descent with Huber Loss")
axes[2].set_xlabel("Step Size")
axes[2].grid(alpha = 0.1)
axes[2].legend()

for key, value in mse_dict2.items():
    axes[3].plot(step_size4, value, label = f"alpha:{key}", alpha = 0.5)
axes[3].set_title("Proximal Gradient Descent")
axes[3].set_xlabel("Step Size")
axes[3].grid(alpha = 0.1)
axes[3].legend()

plt.tight_layout()
plt.show()

In [ ]:
final_weights2, final_mse2 = gradient_descent1(linear_training_data2, linear_testing_data2, gradient_weights3, 0.05)

fig, axes = plt.subplots(1, len(linear_training_data2), figsize = (20, 4))

for i in range(len(linear_training_data2)):
    stand_y_predict = prediction(linear_testing_data2[i][0], final_weights2)
    unstand_y_predict = unstandardise(stand_y_predict, data_mean[12], data_std[12])
    normal_y_predict = np.exp(unstand_y_predict) - 1

    unstand_y_original = unstandardise(linear_testing_data2[i][1], data_mean2[11], data_std2[11])
    normal_y_original = np.exp(unstand_y_original) - 1

    axes[i].plot([j for j in range(normal_y_predict.shape[0])], normal_y_predict, label = "prediction", c = "springgreen")
    axes[i].plot([k for k in range(normal_y_original.shape[0])], normal_y_original, label = "actual", alpha = 0.5, c = "tomato")
    axes[i].legend()

axes[0].set_ylabel("Area")
plt.tight_layout()
plt.show()

print(final_weights2, final_mse2)

In [ ]:
fires2 = pd.read_csv("forestfires.csv")

def buildup_index(dmc, dc):
    if dmc <= 0.4 * dc:
        return 0.8 * ((dmc * dc) / (dmc + (0.4 * dc)))
    else:
        return dmc - (1 - ((0.8 * dc)/(dmc + (0.4 * dc))) * (0.92 + (0.0114 * dmc)**1.7))

def convert_bui(bui):
    if bui <= 80:
        return (0.626 * (bui)**0.809) + 2
    else:
        return 1000 / (25 + (108.64 * (np.exp(-0.023 * bui))))

def b_scale(isi, bui):
    return 0.1 * isi * convert_bui(bui)

def s_scale(isi, bui):
    paramater = b_scale(isi, bui)
    if paramater > 1:
        return np.exp(2.72 * (0.434 * np.log(paramater))**0.647)
    else:
        return paramater

fires2["BUI"] = [buildup_index(dmc, dc) for dmc, dc in zip(fires2["DMC"], fires2["DC"])]
fires2["FWI"] = [s_scale(isi, bui) for isi, bui in zip(fires2["ISI"], fires2["BUI"])]

fires2.to_csv("updated_forestfires.csv", index = False)

In [ ]:
forest_fires_data3 = np.genfromtxt("updated_forestfires.csv", delimiter = ",", skip_header = 1, usecols = [0, 2, 14, 12],
                                  converters = {2: converter_month2, 12: converter_log})

standardised_forest_fires_data3, data_mean3, data_std3 = standardise(forest_fires_data3)
k_fold_training_data3, k_fold_testing_data3 = k_fold_splits(standardised_forest_fires_data3, 5)

In [ ]:
linear_training_data3, linear_testing_data3 = [], []
for i in range(len(k_fold_training_data3)):
    linear_training_data3.append((linear_data(k_fold_training_data3[i][0]), k_fold_training_data3[i][1]))
    linear_testing_data3.append((linear_data(k_fold_testing_data3[i][0]), k_fold_testing_data3[i][1]))

linear_analysis(linear_training_data3, linear_testing_data3)

In [ ]:
poly_analysis(k_fold_training_data3, k_fold_testing_data3, 50)

In [ ]:
ridge_training_data3, ridge_testing_data3 = pre_poly(k_fold_training_data3, k_fold_testing_data3, 25)

optimal_degree3, optimal_alpha3, best_weights_mean3, best_weights_std3, best_mse_mean3, best_mse_std3 = ridge_analysis(ridge_training_data3,
                                                                                                                       ridge_testing_data3,
                                                                                                                       np.linspace(0, 20000, 500))

print(optimal_degree3, optimal_alpha3, best_weights_mean3, best_weights_std3, best_mse_mean3, best_mse_std3)

In [ ]:
gradient_weights5 = np.zeros(linear_training_data3[0][0].shape[1])

step_size1 = np.logspace(-10, -0.4, 51)
gd1_weights_list, gd1_mse_list = [], []
for size in step_size1:
    gd1_weights, gd1_mse = gradient_descent1(linear_training_data3, linear_testing_data3, gradient_weights5, size)
    gd1_weights_list.append(gd1_weights)
    gd1_mse_list.append(gd1_mse)

In [ ]:
gradient_weights6 = np.zeros(ridge_training_data3[optimal_degree3][0][0].shape[1])

step_size2 = np.logspace(-10, -6, 51)
gd2_weights_list, gd2_mse_list = [], []
for size in step_size2:
    gd2_weights, gd2_mse = gradient_descent2(ridge_training_data3[optimal_degree3], ridge_testing_data3[optimal_degree3], gradient_weights6,
                                             optimal_alpha3, size)
    gd2_weights_list.append(gd2_weights)
    gd2_mse_list.append(gd2_mse)

In [ ]:
step_size3 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict1 = {}
for size in step_size3:
    for a in alpha:
        for p in paramater:
            gd3_weights, gd3_mse = huber_loss_gradient_descent(linear_training_data3, linear_testing_data3, gradient_weights5, a, p, size)
            if (a, p) not in mse_dict1:
                mse_dict1[(a, p)] = []
            mse_dict1[(a, p)].append(gd3_mse)

In [ ]:
step_size4 = np.logspace(-10, -0.4, 51) # -0.4
paramater = [0.1, 1, 10]
alpha = [0.001, 0.1]

mse_dict2 = {}
for size in step_size4:
    for a in alpha:
        for p in paramater:
            gd4_weights, gd4_mse = proximal_gradient_descent(linear_training_data3, linear_testing_data3, gradient_weights5, a, p, size)
            if (a, p) not in mse_dict2:
                mse_dict2[(a, p)] = []
            mse_dict2[(a, p)].append(gd4_mse)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize = (20, 4))

axes[0].plot(step_size1, gd1_mse_list)
axes[0].set_title("Linear Gradient Descent")
axes[0].set_xlabel("Step Size")
axes[0].set_ylabel("MSE")
axes[0].grid(alpha = 0.1)

axes[1].plot(step_size2, gd2_mse_list)
axes[1].set_title("Ridge Gradient Descent")
axes[1].set_xlabel("Step Size")
axes[1].grid(alpha = 0.1)

for key, value in mse_dict1.items():
    axes[2].plot(step_size3, value, label = f"(alpha, paramater):{key}", alpha = 0.5)
axes[2].set_title("Gradient Descent with Huber Loss")
axes[2].set_xlabel("Step Size")
axes[2].grid(alpha = 0.1)
axes[2].legend()

for key, value in mse_dict2.items():
    axes[3].plot(step_size4, value, label = f"alpha:{key}", alpha = 0.5)
axes[3].set_title("Proximal Gradient Descent")
axes[3].set_xlabel("Step Size")
axes[3].grid(alpha = 0.1)
axes[3].legend()

plt.tight_layout()
plt.show()

In [ ]:
final_weights3, final_mse3 = gradient_descent2(ridge_training_data3[optimal_degree3], ridge_testing_data3[optimal_degree3], gradient_weights6,
                                               optimal_alpha3, 1.0e-06)

fig, axes = plt.subplots(1, len(ridge_training_data3[optimal_degree3]), figsize = (20, 3.5))

for i in range(len(linear_training_data3)):
    stand_y_predict = prediction(ridge_testing_data3[optimal_degree3][i][0], final_weights3)
    unstand_y_predict = unstandardise(stand_y_predict, data_mean3[3], data_std3[3])
    normal_y_predict = np.exp(unstand_y_predict) - 1

    unstand_y_original = unstandardise(ridge_testing_data3[optimal_degree3][i][1], data_mean3[3], data_std3[3])
    normal_y_original = np.exp(unstand_y_original) - 1

    axes[i].plot([j for j in range(normal_y_predict.shape[0])], normal_y_predict, label = "prediction", c = "springgreen")
    axes[i].plot([k for k in range(normal_y_original.shape[0])], normal_y_original, label = "actual", alpha = 0.5, c = "tomato")
    axes[i].legend()

axes[0].set_ylabel("Area")
plt.tight_layout()
plt.savefig("prediction3.png")
plt.show()

print(final_weights3, final_mse3)